In [1]:
# Add project root to Python path
import sys
import os
from pathlib import Path
import polars as pl

# Find project root from current notebook location
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # When running from examples folder
    project_root = current_dir.parent.parent
else:
    # When running from project root
    project_root = current_dir

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import required modules
from typing import List, Dict, Any, Optional
from datetime import datetime

from quantbt import (
    TradingStrategy,
    BacktestEngine,  
    
    # Basic modules
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # Order related
    Order, OrderSide, OrderType,
)

In [2]:
class MultiSymbolSMAStrategy(TradingStrategy):
    """Volatility-based symbol selection SMA strategy
    
    Hybrid approach:
    - Indicator calculation: Polars vector operations (SMA + volatility)
    - Cross-symbol comparison: Calculate volatility ranking per timestamp in precompute_indicators
    - Signal generation: Dict Native method (trade only highest volatility symbol)
    
    Buy: Price above SMA15 + highest volatility ranking
    Sell: Price below SMA30 + holding position
    """
    
    def __init__(self, buy_sma: int = 15, sell_sma: int = 30, volatility_window: int = 14, symbols: List[str] = ["KRW-BTC", "KRW-ETH"]):
        super().__init__(
            name="VolatilityBasedMultiSymbolSMAStrategy",
            config={
                "buy_sma": buy_sma,
                "sell_sma": sell_sma,
                "volatility_window": volatility_window
            },
            position_size_pct=0.8,  # 80% position size
            max_positions=1
        )
        self.buy_sma = buy_sma
        self.sell_sma = sell_sma
        self.volatility_window = volatility_window
        self.symbols = symbols
        
    def calculate_volatility(self, prices: pl.Series, window: int = 14) -> pl.Series:
        """Calculate volatility based on rolling standard deviation"""
        returns = prices.pct_change()
        return returns.rolling_std(window_size=window)
        
    def _compute_indicators_for_symbol(self, symbol_data):
        """Calculate basic indicators per symbol (Polars vector operations)"""
        
        # Ensure time-sorted order
        data = symbol_data.sort("timestamp")
        
        # Calculate simple moving averages
        buy_sma = self.calculate_sma(data["close"], self.buy_sma)
        sell_sma = self.calculate_sma(data["close"], self.sell_sma)
        
        # Calculate volatility (standard deviation based)
        volatility = self.calculate_volatility(data["close"], self.volatility_window)
        
        # Add indicator columns
        return data.with_columns([
            buy_sma.alias(f"sma_{self.buy_sma}"),
            sell_sma.alias(f"sma_{self.sell_sma}"),
            volatility.alias("volatility")
        ])
    
    # precompute_indicators is automatically executed with standard 2-step processing in BaseStrategy
    # Step 1: _compute_indicators_for_symbol (per-symbol indicators)
    # Step 2: _compute_cross_symbol_indicators (cross-symbol comparison)
    
    def _compute_cross_symbol_indicators(self, data: pl.DataFrame) -> pl.DataFrame:
        """Calculate cross-symbol comparison indicators - ensure time synchronization (complete vector operations)"""
        
        # 🚀 Complete vector operation approach: utilizing window functions
        ranked_data = data.with_columns([
            # Calculate volatility ranking per timestamp (handle None/NaN as inf)
            pl.col("volatility")
            .fill_null(float('inf'))
            .fill_nan(float('inf'))
            .rank("ordinal")
            .over("timestamp")  # Window function per timestamp
            .alias("vol_rank"),
            
            # Determine if it has minimum volatility per timestamp
            (pl.col("volatility") == pl.col("volatility").min().over("timestamp"))
            .alias("is_lowest_volatility")
        ])
        
        return ranked_data
    
    def generate_signals(self, current_data: Dict[str, Any]) -> List[Order]:
        """Generate signals with volatility ranking-based filtering"""
        orders = []
        
        if not self.broker:
            return orders
        
        symbol = current_data['symbol']
        current_price = current_data['close']
        buy_sma = current_data.get(f'sma_{self.buy_sma}')
        sell_sma = current_data.get(f'sma_{self.sell_sma}')
        vol_rank = current_data.get('vol_rank', 999)
        
        # Volatility ranking based filtering (trade only rank 1)
        if vol_rank != 1:
            return orders  # Stop trading if not highest volatility
        
        # Skip if indicators are not calculated
        if buy_sma is None or sell_sma is None:
            return orders
        
        current_positions = self.get_current_positions()
        
        # Buy signal: Price above SMA15 + No position + Highest volatility
        if current_price > buy_sma and symbol not in current_positions:
            portfolio_value = self.get_portfolio_value()
            quantity = self.calculate_position_size(symbol, current_price, portfolio_value) / len(self.symbols)
            
            if quantity > 0:
                order = Order(
                    symbol=symbol,
                    side=OrderSide.BUY,
                    quantity=quantity,
                    order_type=OrderType.MARKET
                )
                orders.append(order)
                # print(f"🎯 Highest volatility buy: {symbol} @ {current_price:,.0f} (SMA{self.buy_sma}: {buy_sma:,.0f}, Vol rank: {vol_rank})")
        
        # Sell signal: Price below SMA30 + Has position (regardless of volatility ranking)
        elif current_price < sell_sma and symbol in current_positions and current_positions[symbol] > 0:
            order = Order(
                symbol=symbol,
                side=OrderSide.SELL,
                quantity=current_positions[symbol],
                order_type=OrderType.MARKET
            )
            orders.append(order)
            # print(f"📉 Sell signal: {symbol} @ {current_price:,.0f} (SMA{self.sell_sma}: {sell_sma:,.0f})")
        
        return orders


In [3]:


# 1. Upbit data provider
print("🔄 Initializing data provider...")
upbit_provider = UpbitDataProvider()

# 2. Backtesting configuration
config = BacktestConfig(
    symbols=["KRW-BTC", 'KRW-ETH'],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31), 
    timeframe="1d",  # Daily bars (faster than 1-minute bars)
    initial_cash=10_000_000,  # 10 million KRW
    commission_rate=0.0,      # 0% commission (for testing) - use appropriate value for actual backtesting
    slippage_rate=0.0,         # 0% slippage (for testing) - use appropriate value for actual backtesting
    save_portfolio_history=True
)

# 3. SMA strategy
print("⚡ Initializing strategy...")
strategy = MultiSymbolSMAStrategy(
    buy_sma=15,   # Buy: Price above 15-day moving average
    sell_sma=30,   # Sell: Price below 30-day moving average
    symbols=["KRW-BTC", "KRW-ETH"]
)

# 4. Broker configuration
broker = SimpleBroker(
    initial_cash=config.initial_cash,
    commission_rate=config.commission_rate,
    slippage_rate=config.slippage_rate
)

# 5. Dict Native backtest engine (Phase 7)
print("🚀 Initializing Dict Native backtest engine...")
engine = BacktestEngine()  # Using Dict Native engine!
engine.set_strategy(strategy)
engine.set_data_provider(upbit_provider)
engine.set_broker(broker)

🔄 Initializing data provider...
⚡ Initializing strategy...
🚀 Initializing Dict Native backtest engine...


In [4]:
# 7. Results output
result = engine.run(config)
    
# Print result summary
result.print_summary()

처리중... 370/730:  51%|█████     | 370/730 [00:00<00:00]

처리중... 730/730: 100%|██████████| 730/730 [00:00<00:00]

                 BACKTEST RESULTS SUMMARY
Period          : 2024-01-01 ~ 2024-12-31
Initial Capital : $10,000,000
Final Equity    : $16,452,878
Total Return    : 64.53%
Annual Return   : 64.58%
Volatility      : 23.73%
Sharpe Ratio    : 2.72
Calmar Ratio    : 3.29
Sortino Ratio   : 3.57
Max Drawdown    : 19.65%
Total Trades    : 28
Win Rate        : 57.1%
Profit Factor   : 6.01
Execution Time  : 0.11s


In [5]:
# 1. Portfolio performance chart
result.plot_portfolio_performance()

# 2. Returns distribution 
result.plot_returns_distribution(period="daily")

# 3. Monthly returns heatmap
result.plot_monthly_returns_heatmap()

# 4. Performance comparison table
result.show_performance_comparison()

,Metric,Strategy,Benchmark
0,Total Return (%),64.53,134.35
1,Annual Return (%),64.58,134.49
2,Volatility (%),23.73,46.56
3,Sharpe Ratio,2.72,2.89
4,Calmar Ratio,3.29,4.59
5,Sortino Ratio,3.57,5.06
6,Max Drawdown (%),19.65,29.28
7,Beta,0.38,1.00
8,Alpha,0.14,0.00
9,Total Trades,28,-
